### CommaSeparatedListOutputParser

In [15]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [16]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)
prompt

PromptTemplate(input_variables=['subject'], input_types={}, partial_variables={'format_instructions': 'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'}, template='List five {subject}.\n{format_instructions}')

In [17]:
# OpenAI 모델 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

# "AI 관련 기술"에 대한 체인 호출 실행
result = chain.invoke({"subject": "AI 관련 기술"})

# 쉼표로 구분된 리스트 출력
print(" AI 관련 기술 목록:")
print(result)

# 결과 활용 예시: CSV 파일로 저장
csv_filename = "./data/ai_technologies.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["AI 기술"])  # 헤더 추가
    for item in result:
        writer.writerow([item])

print(f" '{csv_filename}' 파일로 저장 완료!")

 AI 관련 기술 목록:
['Machine Learning', 'Deep Learning', 'Natural Language Processing', 'Computer Vision', 'Robotics']
 './data/ai_technologies.csv' 파일로 저장 완료!


### JsonOutputParser

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import json

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학 분야 전문가 AI입니다. 질문에 대해 체계적이고 간결한 답변을 JSON 형식으로 제공하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# JSON 출력 형식 지침을 프롬프트에 적용
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

In [19]:
# OpenAI 모델 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
# 프롬프트, 모델, 출력 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 질문 설정 (우주 탐사 관련 질문)
question = "최근 10년간 진행된 주요 우주 탐사 미션 3가지를 알려주세요. \
각 미션의 이름은 `mission_name`에, 목표는 `goal`에, 주관 기관은 `agency`에 담아 주세요."

# 체인 실행 및 JSON 응답 받기
response = chain.invoke({"question": question})

# JSON 데이터 출력
print(json.dumps(response, indent=4, ensure_ascii=False))

[
    {
        "mission_name": "뉴호라이즌스(New Horizons)",
        "goal": "명왕성 및 카이퍼 벨트 탐사",
        "agency": "나사(NASA)"
    },
    {
        "mission_name": "화성 탐사선 큐리오시티(Curiosity Rover)",
        "goal": "화성 표면 탐사 및 생명체 존재 여부 조사",
        "agency": "나사(NASA)"
    },
    {
        "mission_name": "창어 4호(Chang'e 4)",
        "goal": "달 뒷면 탐사",
        "agency": "중국 국가우주국(CNSA)"
    }
]


### PandasDataFrameOutputParser

In [20]:
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import re

# Titanic 데이터셋 로드
df = pd.read_csv('data/titanic.csv')

# Pandas DataFrame Output Parser 설정
parser = PandasDataFrameOutputParser(dataframe=df)

# 형식 지침 출력
format_instructions = parser.get_format_instructions()
print("Format Instructions:\n", format_instructions)

Format Instructions:
 The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2,

In [21]:
# ChatOpenAI 모델 초기화
# model = ChatOpenAI(temperature=0, model_name="gpt-4o")
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template=""" 
    You are a helpful assistant that interacts with a Pandas DataFrame.
    The DataFrame contains the following columns: {columns}.
    
    Your task is to answer the user's query by generating a command in the following format:
    {format_instructions}
    
    User Query: {query}    
    """,
    input_variables=["query"],
    partial_variables={
        "format_instructions": format_instructions,
        "columns": ", ".join(df.columns)
    },
)
print(prompt.partial_variables['columns'])

Survived, Pclass, Name, Sex, Age, Siblings/Spouses Aboard, Parents/Children Aboard, Fare


In [22]:
# 체인 생성
chain = prompt | model | parser

# 모델 응답 받기
try:
    # **Name 열을 표시하십시오.**
    print('Name 컬럼 출력')
    df_query = "Show the Name column"

    parser_output = chain.invoke({"query": df_query})
    print(type(parser_output))
    print(parser_output)

    # **첫번째 행을 표시하십시오.**
    print('첫번째 행 출력')
    df_query2 = "Show first row"

    parser_output2 = chain.invoke({"query": df_query2})
    print(parser_output2)

except Exception as e:
    print(f"오류 발생: {e}")

Name 컬럼 출력
오류 발생: Request 'The command in the required format is: 
column:Name. 

This will display the 'Name' column of the DataFrame. 

If you would like to see a specific row or a range of rows, you can modify it as follows:
- To get a specific row: row:1[Name]
- To get a column for specific rows: column:Name[1,3,5]
- To get a column for a range of rows: column:Name[1..3] 
- To perform an operation: mean:Name is not valid since mean is not a string column, but  "mean:Age" or  "sum:Fare" could be used.' is not correctly formatted.                     Please refer to the format instructions.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


In [23]:
import pandas as pd
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# OpenAI 모델 초기화
# model = ChatOpenAI(temperature=0, model_name="gpt-4o")
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 응답 스키마 정의 {data : [{},{},{}] }
response_schemas = [
    ResponseSchema(name="data", description="A list of dictionaries representing table rows."),
]

# Output Parser 설정
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""
    You are an AI assistant that generates tabular data. 
    You must return the data in JSON format that follows this schema:
    
    {format_instructions}
        
    **User Query:**
    {query}
    """,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"data": string  // A list of dictionaries representing table rows.\n}\n```'} template='\n    You are an AI assistant that generates tabular data. \n    You must return the data in JSON format that follows this schema:\n\n    {format_instructions}\n\n    **User Query:**\n    {query}\n    '


In [25]:
# 체인 생성 (프롬프트 → 모델 → OutputParser)
chain = prompt | model | parser

# 실행 함수
def generate_dataframe(user_query):
    try:
        # 모델 호출
        json_response = chain.invoke({"query": user_query})
        print(json_response)
        
        # 모델이 반환한 JSON을 Pandas DataFrame으로 변환
        df = pd.DataFrame(json_response["data"])

        # 결과 출력
        print("\n🔹 Generated DataFrame:\n")
        return df

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None

In [ ]:
# [예제 1] 2024년 상반기 서울 아파트 평균 매매 가격 데이터 생성
print('2024년 하반기 서울 아파트 평균 매매 가격 데이터 생성')
df_seoul_housing = generate_dataframe(
    "Create a dataset of the average apartment sale prices in Seoul for the second half of 2024 with columns: District (구), Average Price (in KRW), Number of Transactions, and Year-over-Year Change (%)."
)
print(df_seoul_housing.shape)
df_seoul_housing